# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [2]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [3]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [41]:
sinasc_raw.ORIGEM.sum()

27028

# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [4]:
sinasc1 = sinasc_raw[['IDADEMAE','IDADEPAI','munResNome']]
sinasc1.head(3)

,IDADEMAE,IDADEPAI,munResNome
0,19,26.0,Alta Floresta D'Oeste
1,29,24.0,Alta Floresta D'Oeste
2,37,32.0,Alta Floresta D'Oeste


In [5]:
sinasc1.IDADEPAI.isna().sum()

19421

In [6]:
sinasc1 = sinasc1.dropna(subset=['IDADEPAI'])

In [7]:
sinasc1.IDADEPAI.isna().sum()

0

In [8]:
sinasc1.IDADEPAI.isnull().sum()

0

In [9]:
sinasc1.IDADEMAE.isna().sum()

0

In [10]:
result1 = sinasc1.groupby('munResNome')[['IDADEMAE','IDADEPAI']].mean()

In [14]:
result1_mae = result1.IDADEMAE.sort_values()
result1_mae.head(5)

munResNome
Alto Paraíso               22.833333
Candeias do Jamari         23.684211
Município ignorado - RO    24.000000
Alto Alegre dos Parecis    24.679104
Buritis                    24.750000
Name: IDADEMAE, dtype: float64

In [15]:
result1_pai = result1.IDADEPAI.sort_values()
result1_pai.head(5)

munResNome
Município ignorado - RO      22.000000
Rio Crespo                   28.000000
Alto Paraíso                 28.833333
Alto Alegre dos Parecis      29.179104
Governador Jorge Teixeira    29.222222
Name: IDADEPAI, dtype: float64

### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [19]:
sinasc2 = sinasc_raw[['SEXO','ESCMAE','DTNASC','PESO']]
sinasc2.head(5)

,SEXO,ESCMAE,DTNASC,PESO
0,Masculino,8 a 11 anos,2019-02-19,3685
1,Feminino,8 a 11 anos,2019-02-21,3055
2,Feminino,8 a 11 anos,2019-02-25,3460
3,Feminino,12 anos ou mais,2019-03-20,2982
4,Feminino,8 a 11 anos,2019-03-23,3310


In [26]:
#sinasc2 = sinasc2[sinasc2['DTNASC'] == '2019-09-17'] uma maneira de fazer o filtro porém a de baixo serviria caso houvesse anos diferentes
sinasc2 = sinasc2[sinasc2['DTNASC'].str.contains('-09-17')]

In [28]:
sinasc2.head(5)

,SEXO,ESCMAE,DTNASC,PESO
13788,Feminino,8 a 11 anos,2019-09-17,3195
14278,Masculino,8 a 11 anos,2019-09-17,2965
14316,Masculino,8 a 11 anos,2019-09-17,3085
14317,Masculino,12 anos ou mais,2019-09-17,3655
14436,Feminino,8 a 11 anos,2019-09-17,3410


In [37]:
sinasc2.DTNASC.nunique()

1

In [35]:
sinasc2.ESCMAE.unique()

array(['8 a 11 anos', '12 anos ou mais', '4 a 7 anos', nan], dtype=object)

In [30]:
result2 = sinasc2.groupby(['SEXO','ESCMAE'])['PESO'].mean()
result2

SEXO       ESCMAE         
Feminino   12 anos ou mais    3143.333333
           4 a 7 anos         2932.857143
           8 a 11 anos        3063.000000
Masculino  12 anos ou mais    3446.000000
           4 a 7 anos         3556.875000
           8 a 11 anos        3235.333333
Name: PESO, dtype: float64

### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [55]:
mim_bbs = sinasc_raw[['munResNome','IDADEMAE','IDADEPAI','ORIGEM']]
mim_bbs.head(5)

,munResNome,IDADEMAE,IDADEPAI,ORIGEM
0,Alta Floresta D'Oeste,19,26.0,1
1,Alta Floresta D'Oeste,29,24.0,1
2,Alta Floresta D'Oeste,37,32.0,1
3,Alto Alegre dos Parecis,30,24.0,1
4,Alta Floresta D'Oeste,30,27.0,1


In [56]:
mim_bbs.ORIGEM.unique()

array([1], dtype=int64)

In [58]:
result3 = mim_bbs.groupby('munResNome')['ORIGEM'].sum()
result3.sort_values(ascending= True).head(5)

munResNome
Município ignorado - RO     1
Castanheiras               32
Pimenteiras do Oeste       40
Primavera de Rondônia      43
Parecis                    44
Name: ORIGEM, dtype: int64

### Qual o municipio que nasceu menos bebe em 2019?
R: Castanheiras

In [73]:
mim_bbs_cast = mim_bbs[mim_bbs['munResNome'] == 'Castanheiras']
mim_bbs_cast.head(3)

,munResNome,IDADEMAE,IDADEPAI,ORIGEM
2067,Castanheiras,18,17.0,1
2300,Castanheiras,31,36.0,1
4512,Castanheiras,22,NaN,1


In [75]:
#mim_bbs_cast = mim_bbs_cast.dropna()
mim_bbs_cast.head(3)

,munResNome,IDADEMAE,IDADEPAI,ORIGEM
2067,Castanheiras,18,17.0,1
2300,Castanheiras,31,36.0,1
4512,Castanheiras,22,NaN,1


In [77]:
#mim_bbs_cast.agg({
#    'IDADEMAE': [('mínimo', 'min'), ('maximo', 'max'), ('media', 'mean')],
#    'IDADEPAI': [('mínimo', 'min'), ('maximo', 'max'), ('media', 'mean')]
#})
mim_bbs_cast.agg({
    'IDADEMAE': ['min', 'max', 'mean'],
    'IDADEPAI': ['min', 'max', 'mean']
})

,IDADEMAE,IDADEPAI
min,17.00000,17.000000
max,39.00000,43.000000
mean,27.28125,30.392857


### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [83]:
sinasc3 = sinasc_raw[['munResNome','IDADEMAE','IDADEPAI','ORIGEM','QTDFILVIVO','ORIGEM','DTNASC']]

In [85]:
max_bbs_marc = sinasc3[sinasc3['DTNASC'].str.contains('-03-')]
max_bbs_marc

,munResNome,IDADEMAE,IDADEPAI,ORIGEM,QTDFILVIVO,ORIGEM,DTNASC
3,Alto Alegre dos Parecis,30,24.0,1,0.0,1,2019-03-20
4,Alta Floresta D'Oeste,30,27.0,1,1.0,1,2019-03-23
5,Alta Floresta D'Oeste,33,40.0,1,2.0,1,2019-03-27
9,Alta Floresta D'Oeste,30,41.0,1,1.0,1,2019-03-11
10,Alta Floresta D'Oeste,31,28.0,1,1.0,1,2019-03-11
...,...,...,...,...,...,...,...
13670,Porto Velho,24,NaN,1,0.0,1,2019-03-21
13671,Porto Velho,24,NaN,1,2.0,1,2019-03-22
13736,Porto Velho,35,42.0,1,1.0,1,2019-03-07
13748,Vilhena,36,NaN,1,2.0,1,2019-03-30


In [89]:
result4 = max_bbs_marc.groupby('munResNome')['ORIGEM'].sum()
result4.sort_values(by='munResNome', ascending= False).head(5)

,ORIGEM,ORIGEM
munResNome,,
Vilhena,148,148
Vale do Paraíso,13,13
Vale do Anari,11,11
Urupá,14,14
Theobroma,8,8


#### Qual o municipio que nasceu mais bebe no mês de março?
R: Vilhena

In [91]:
result5 = max_bbs_marc[max_bbs_marc['munResNome'] == 'Vilhena']
result5.head(3)

,munResNome,IDADEMAE,IDADEPAI,ORIGEM,QTDFILVIVO,ORIGEM,DTNASC
1798,Vilhena,31,32.0,1,0.0,1,2019-03-07
2457,Vilhena,21,30.0,1,1.0,1,2019-03-13
6970,Vilhena,19,NaN,1,NaN,1,2019-03-07


In [92]:
result5.agg({
    'QTDFILVIVO': ['min', 'max', 'mean'],
    'IDADEPAI': ['min', 'max', 'mean'],
    'IDADEMAE': ['min', 'max', 'mean']
})

,QTDFILVIVO,IDADEPAI,IDADEMAE
min,0.000000,27.0000,16.000000
max,5.000000,47.0000,39.000000
mean,0.979592,33.5625,26.412162


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

In [93]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [94]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [95]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

- Respsota

In [96]:
result3.sort_values(ascending= False).head(5)

munResNome
Porto Velho    8437
Ji-Paraná      2182
Ariquemes      1729
Vilhena        1590
Cacoal         1374
Name: ORIGEM, dtype: int64

Percebi uma diferença significativa no número de nascimentos entre a capital e as outras cidades do estado.

In [97]:
result2

SEXO       ESCMAE         
Feminino   12 anos ou mais    3143.333333
           4 a 7 anos         2932.857143
           8 a 11 anos        3063.000000
Masculino  12 anos ou mais    3446.000000
           4 a 7 anos         3556.875000
           8 a 11 anos        3235.333333
Name: PESO, dtype: float64

Também analisei que os bebês do sexo masculino, em média, tendem a nascer com um peso maior do que os do sexo feminino. Além disso, observei que, quanto maior a escolaridade da mãe, maior é a média de peso dos bebês do sexo feminino. Uma curiosidade que percebi foi que, na data do meu nascimento, nenhuma mãe possuía escolaridade inferior a 4 anos.